# NYC Taxi Trip-Duration Estimation

NOTE: This step is only needed if you haven't already

In [144]:
!pip install kaskada

You should consider upgrading via the '/Users/ryan.michael/.pyenv/versions/3.10.5/bin/python3.10 -m pip install --upgrade pip' command.


In [11]:
!pip install matplotlib seaborn xgboost scikit-learn

You should consider upgrading via the '/Users/ryan.michael/.pyenv/versions/3.10.5/bin/python3.10 -m pip install --upgrade pip' command.


1. Create a local, in-memory Kaskada instance

In [1]:
from kaskada.api.session import LocalBuilder
from kaskada import table

session = LocalBuilder().build()

INFO:kaskada.api.release:Using latest release version: engine@v0.8.3
INFO:kaskada.api.release:Skipping download. Using binary: /Users/ryan.michael/.cache/kaskada/bin/engine@v0.8.3/kaskada-engine
INFO:kaskada.api.release:Skipping download. Using binary: /Users/ryan.michael/.cache/kaskada/bin/engine@v0.8.3/kaskada-manager
INFO:kaskada.api.local_session.local_service:Initializing manager process
INFO:kaskada.api.local_session.local_service:Logging manager STDOUT to /Users/ryan.michael/.cache/kaskada/logs/2023-07-17T12-25-12-manager-stdout.log
INFO:kaskada.api.local_session.local_service:Logging manager STDERR to /Users/ryan.michael/.cache/kaskada/logs/2023-07-17T12-25-12-manager-stdout.log
INFO:kaskada.api.local_session.local_service:Initializing engine process
INFO:kaskada.api.local_session.local_service:Logging engine STDOUT to /Users/ryan.michael/.cache/kaskada/logs/2023-07-17T12-25-12-engine-stdout.log
INFO:kaskada.api.local_session.local_service:Logging engine STDERR to /Users/ryan.m

In [2]:
%load_ext fenlmagic

INFO:fenlmagic:extension loaded


## Data Prep

We'll be using data originally obtained from https://chriswhong.com/open-data/foil_nyc_taxi/. Recent datasets provided by NYC use more granular geographical location, rather than the fine-grained latitude and longitude data present in the original dataset released in 2013.

The data has been cleaned by converting from CSV to Parquet, and then the "Trip" and "Fare" datasets have been combined with a simple join:

```sql
copy (
    select * from 'trip_data.parquet' 
    join (select * from 'trip_fare.parquet') as fares 
    on  trip_data.medallion = fares.medallion
    and trip_data.hack_license = fares.hack_license 
    and trip_data.pickup_datetime = fares.pickup_datetime
    and trip_data.vendor_id = fares.vendor_id
) to 'combined.parquet' (FORMAT PARQUET)
```

Both the trip and fare data contain 173,179,759 rows, but the join result contains 173,185,091 (5,332 more) rows. These rows are presumed to be duplicates and are ignored for this analysis, as they contitute 0.003% of the full dataset.

Since the dataset contains timestamps for both pickup and dropoff time, the dataset has been split into separate files, one describing pickup events and another containing dropoff events. The fields present in the pickup dataset are filtered to information that could plausibly be known at pickup time, specifically the following fields are omitted:

* rate_code: Rate code is the "final rate code in effect at the end of the trip"
* store_and_fwd_flag: Potentially reflects information about the route take, for example tunnels.
* dropoff_datetime: The trip duration can't be known in advance
* trip_time_in_secs: See above
* trip_distance: See above
* payment_type: Payment is made at the end of the trip
* fare_amount: The fare amount depends on the trip duration
* surcharge: Surcharge depends on the fare amount
* mta_tax: See above
* tip_amount: Tips are recorded at the end of a ride
* tolls_amount: Toll amounts depend on the route taken
* total_amount: Total depends on multiple fields not known in advance

Specifically, the pickup dataset is created from the query:

```sql
copy (
    select medallion, hack_license, vendor_id, pickup_datetime, passenger_count, pickup_longitude, pickup_latitude,
    from 'combined.parquet'
) to 'pickups.parquet' (FORMAT PARQUET)
```

2. Create a table for the data

In [5]:
#table.delete_table("Pickup")
table.create_table(
  # The table's name
  table_name = "Pickup",
  # The name of the column in the data that contains the time associated with each row
  time_column_name = "pickup_datetime",
  # The name of the column in the data that contains the entity key associated with each row
  entity_key_column_name = "hack_license",
  grouping_id = "License",
)

table {
  table_id: "cbd6b01d-9e84-4718-afca-2fa69afe6ad5"
  table_name: "Pickup"
  time_column_name: "pickup_datetime"
  entity_key_column_name: "hack_license"
  grouping_id: "License"
  create_time {
    seconds: 1689611138
    nanos: 959169000
  }
  update_time {
    seconds: 1689611138
    nanos: 959169000
  }
  source {
    kaskada {
    }
  }
}
request_details {
  request_id: "52d6e19599188666a3299e2d2c531f0d"
}

In [6]:
#table.delete_table("Dropoff")
table.create_table(
  # The table's name
  table_name = "Dropoff",
  # The name of the column in the data that contains the time associated with each row
  time_column_name = "dropoff_datetime",
  # The name of the column in the data that contains the entity key associated with each row
  entity_key_column_name = "hack_license",
  grouping_id = "License",
)

table {
  table_id: "23b84cee-bc49-4e39-b8f0-c54d308857d8"
  table_name: "Dropoff"
  time_column_name: "dropoff_datetime"
  entity_key_column_name: "hack_license"
  grouping_id: "License"
  create_time {
    seconds: 1689611145
    nanos: 783391000
  }
  update_time {
    seconds: 1689611145
    nanos: 783391000
  }
  source {
    kaskada {
    }
  }
}
request_details {
  request_id: "ed5c5088509fbf18054352b813874508"
}

3. Load the files's contents into the Purchase table

In [7]:
table.load(table_name = "Pickup", file = "pickups_100000.parquet")

data_token_id: "4e741874-3981-4fca-a527-841620828741"
request_details {
  request_id: "8a5d0cb3520fef68ffc496ec742241f3"
}

In [8]:
table.load(table_name = "Dropoff", file = "combined_100000.parquet")

data_token_id: "7b526b0a-69e7-4ee2-96fc-f78bd654d5ef"
request_details {
  request_id: "4c6cb123f6d2393b97ddc337de7fbf3e"
}

In [11]:
from kaskada.slice_filters import EntityPercentFilter
import kaskada.client
filter_percentage = 0.1
entity_filter = EntityPercentFilter(filter_percentage)
kaskada.client.set_default_slice(entity_filter)

In [13]:
%%fenl --var all
Pickup

,_time,_subsort,_key_hash,_key,medallion,hack_license,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_latitude,dropoff_longitude
0,2013-01-01 00:00:00,13448911311440400776,15913485641726903213,33276CA24A915CBD668AF96873D07883,08E54F4C460720DDE43460E354486FBC,33276CA24A915CBD668AF96873D07883,CMT,2013-01-01 00:00:00,1,-73.999878,40.743343,40.748280,-74.003708
1,2013-01-01 00:00:00,13448911311440400777,9006261832047477220,25BA06A87905667AA1FE5990E33F0E2E,76942C3205E17D7E7FE5A9F709D16434,25BA06A87905667AA1FE5990E33F0E2E,VTS,2013-01-01 00:00:00,3,-73.955925,40.781887,40.777832,-73.963181
2,2013-01-01 00:00:00,13448911311440400778,7881235434597582542,252392FA3463A93EC0D0F81A4EE2D05B,5C7501130D5FBCA3C65B6C6A5554D164,252392FA3463A93EC0D0F81A4EE2D05B,VTS,2013-01-01 00:00:00,6,-74.006927,40.740765,40.739616,-73.982994
3,2013-01-01 00:00:00,13448911311440400779,1043742014481570536,CABFA7E28A15A656CF1B30D141B72414,18F11AB7B8E48739B14DA4C4314F99FC,CABFA7E28A15A656CF1B30D141B72414,VTS,2013-01-01 00:00:00,1,-73.995056,40.760185,40.754955,-73.968590
4,2013-01-01 00:00:00,13448911311440400780,10106009620658921357,BB899DFEA9CC964B50C540A1D685CCFB,468244D1361B8A3EB8D206CC394BC9E9,BB899DFEA9CC964B50C540A1D685CCFB,VTS,2013-01-01 00:00:00,1,-73.955383,40.779728,40.760326,-73.967758
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2013-01-01 03:22:00,13448911311440500771,9094629955177181826,934F55A3C4D15C6AAEA16710A42E297D,BE386D8524FCD16B3727DCF0A32D9B25,934F55A3C4D15C6AAEA16710A42E297D,VTS,2013-01-01 03:22:00,2,-73.937996,40.803677,40.859760,-73.929893
99996,2013-01-01 03:22:00,13448911311440500772,18372752153190913550,0FEB05AD962BB9F20DE34A02283DF5FB,390CF993D59D0DC7297CE2A0BF73D4F5,0FEB05AD962BB9F20DE34A02283DF5FB,VTS,2013-01-01 03:22:00,2,-73.973160,40.792767,40.758629,-73.959747
99997,2013-01-01 03:22:00,13448911311440500773,4952570644952036406,B2B964896730C55143F4261F988E58B5,9730AFBE8E88604CA7BE8224C3D06F16,B2B964896730C55143F4261F988E58B5,VTS,2013-01-01 03:22:00,5,-73.940659,40.715668,40.661446,-73.982803
99998,2013-01-01 03:22:00,13448911311440500774,12766428500220350347,95EA78362BDBD8561F3E9B53E50A0B26,CA2E404BE3197B507CBBA41D1E0EA70F,95EA78362BDBD8561F3E9B53E50A0B26,VTS,2013-01-01 03:22:00,2,-73.980217,40.722229,40.690746,-73.957146


## Explore!

Try some queries of your own in the block below. The [Reference > Queries](https://kaskada.io/docs-site/kaskada/main/developing/queries.html) section of the docs can help you get started.

Predict
* Fare - https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction/overview
* Duration - https://www.kaggle.com/c/nyc-taxi-trip-duration
* Wait time
* Tip amount <<<

* Time of day
* Distance
* year
* day of week

Feature engineering based on
* https://www.kaggle.com/code/headsortails/nyc-taxi-eda-update-the-fast-the-curious
* https://www.kaggle.com/code/maheshdadhich/strength-of-visualization-python-visuals-tutorial

In [14]:
%%fenl --var examples

# TODO: GIS functions so we can compute actual distances from latitude / longitude
# This just takes the euclidean distance - probably OK for this example but terrible 
# at larger scales or higher latitudes.
let distance = sqrt(
    Pickup.pickup_latitude - Pickup.dropoff_latitude | powf(power=2) +
    Pickup.pickup_longitude - Pickup.dropoff_longitude | powf(power=2)) | else(-1)

# TODO: GIS bucketing a la H3.
# This just normalizes to reasonable extents for NYC 
# Latitude: [40.650, 40.850] => [0, 200]
# Longitude: [-74.05, -73.75] => [0, 200]
let pu_src_lat_bin = (((Pickup.pickup_latitude - 40.65) * (200/0.2)) as i64)  | else(-1)
let do_src_lat_bin = (((Dropoff.pickup_latitude - 40.65) * (200/0.2)) as i64)  | last()
let pu_src_lon_bin = ((Pickup.pickup_longitude + 74.05) * (200/0.3)) as i64
let do_src_lon_bin = (((Dropoff.pickup_longitude + 74.05) * (200/0.3)) as i64) | last()
let do_dst_lat_bin = (((Dropoff.dropoff_latitude - 40.65) * (200/0.2)) as i64)  | last()
let do_dst_lon_bin = (((Dropoff.dropoff_longitude + 74.05) * (200/0.3)) as i64) | last()

# TODO: Allow record-typed entity keys
# For now, this just shifts the latitude and adds it to the longitude.
# This produces an identifier, but it's not ideal.
let pu_src_bin_id = (pu_src_lat_bin + 1000) + pu_src_lon_bin
let do_src_bin_id = (do_src_lat_bin + 1000) + do_src_lon_bin

# Compute some per-trip metrics for Dropoffs
let trip_dist = sqrt(
        Dropoff.pickup_latitude - Dropoff.dropoff_latitude | powf(power=2) +
        Dropoff.pickup_longitude - Dropoff.dropoff_longitude | powf(power=2))
let trip_speed = 100000 * trip_dist / Dropoff.trip_time_in_secs
let dropoff_with_metrics = Dropoff | extend({trip_dist, trip_speed})

# Re-key by trip source and destination
let dropoff_by_src_bin = dropoff_with_metrics | with_key(pu_src_bin_id)
let dropoff_by_dst_bin = dropoff_with_metrics | with_key(do_src_bin_id)

# Compute aggregates related to trips starting and ending in various locations
let src_bin_mean_speed_10m = dropoff_by_src_bin.trip_speed | mean(window=sliding(10,minutely()))
let src_bin_mean_speed_30m = dropoff_by_src_bin.trip_speed | mean(window=sliding(30,minutely()))
let src_bin_mean_speed_60m = dropoff_by_src_bin.trip_speed | mean(window=sliding(60,minutely()))
let dst_bin_mean_speed_10m = dropoff_by_dst_bin.trip_speed | mean(window=sliding(10,minutely()))
let dst_bin_mean_speed_30m = dropoff_by_dst_bin.trip_speed | mean(window=sliding(30,minutely()))
let dst_bin_mean_speed_60m = dropoff_by_dst_bin.trip_speed | mean(window=sliding(60,minutely()))
let src_dropoff_count_10m = dropoff_by_src_bin | count(window=sliding(10, minutely()))
let src_dropoff_count_30m = dropoff_by_src_bin | count(window=sliding(30, minutely()))
let src_dropoff_count_60m = dropoff_by_src_bin | count(window=sliding(60, minutely()))
let dst_dropoff_count_10m = dropoff_by_dst_bin | count(window=sliding(10, minutely()))
let dst_dropoff_count_30m = dropoff_by_dst_bin | count(window=sliding(30, minutely()))
let dst_dropoff_count_60m = dropoff_by_dst_bin | count(window=sliding(60, minutely()))

in {
    # hour of day
    # day of week
    # geo cluster
    monthday: day_of_month(Pickup.pickup_datetime as timestamp_ns) | else(-1),
    distance,
    pu_src_lat_bin,
    pu_src_lon_bin,
    src_bin_mean_speed_10m: src_bin_mean_speed_10m | lookup(pu_src_bin_id) | else (-1),
    src_bin_mean_speed_30m: src_bin_mean_speed_30m | lookup(pu_src_bin_id) | else (-1),
    src_bin_mean_speed_60m: src_bin_mean_speed_60m | lookup(pu_src_bin_id) | else (-1),
    dst_bin_mean_speed_10m: dst_bin_mean_speed_10m | lookup(pu_src_bin_id) | else (-1),
    dst_bin_mean_speed_30m: dst_bin_mean_speed_30m | lookup(pu_src_bin_id) | else (-1),
    dst_bin_mean_speed_60m: dst_bin_mean_speed_60m | lookup(pu_src_bin_id) | else (-1),
    src_bin_dropoffs_10m: src_dropoff_count_10m | lookup(pu_src_bin_id) | else (-1),
    src_bin_dropoffs_30m: src_dropoff_count_30m | lookup(pu_src_bin_id) | else (-1),
    src_bin_dropoffs_60m: src_dropoff_count_60m | lookup(pu_src_bin_id) | else (-1),
    dst_bin_dropoffs_10m: dst_dropoff_count_10m | lookup(pu_src_bin_id) | else (-1),
    dst_bin_dropoffs_30m: dst_dropoff_count_30m | lookup(pu_src_bin_id) | else (-1),
    dst_bin_dropoffs_60m: dst_dropoff_count_60m | lookup(pu_src_bin_id) | else (-1),
    passengers: Pickup.passenger_count | else(-1),
    vendor: Pickup.vendor_id,
} 
| when(is_valid(Pickup))
| last(window=since(is_valid(Dropoff)))
| when(is_valid(Dropoff))
| extend({target: Dropoff.trip_time_in_secs})

# cleaning
| when($input.distance < 0.5) # distance outliers
| when($input.distance > 0) # very short trips
| when($input.target < 24 * 60 * 60) # trips longer than a day
| when($input.target > 60) # trips shorter than a minute

,_time,_subsort,_key_hash,_key,target,monthday,distance,pu_src_lat_bin,pu_src_lon_bin,src_bin_mean_speed_10m,src_bin_mean_speed_30m,src_bin_mean_speed_60m,dst_bin_mean_speed_10m,dst_bin_mean_speed_30m,dst_bin_mean_speed_60m,src_bin_dropoffs_10m,src_bin_dropoffs_30m,src_bin_dropoffs_60m,dst_bin_dropoffs_10m,dst_bin_dropoffs_30m,dst_bin_dropoffs_60m,passengers,vendor
0,2013-01-01 00:01:22,3541431521954618907,9363641051848511460,77FFDF38272A6006517D53EDA14333E2,61,1,0.000328,118,53,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,0,0,0,-1,-1,-1,1,CMT
1,2013-01-01 00:01:37,3541431521954618900,9335815436245950131,CDCB7729DE07243726FF7BB0BD5D06BF,83,1,0.001897,99,49,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,0,0,0,-1,-1,-1,1,CMT
2,2013-01-01 00:02:00,3541431521954618881,3086613677724504812,E7750A37CAB07D0DFF0AF7E3573AC141,120,1,0.005916,66,62,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,0,0,0,-1,-1,-1,1,VTS
3,2013-01-01 00:02:44,3541431521954618890,15913485641726903213,33276CA24A915CBD668AF96873D07883,163,1,0.003854,93,33,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,0,0,0,-1,-1,-1,1,CMT
4,2013-01-01 00:03:00,3541431521954618852,16896641161375836099,B6B300DB052529A0A9A238DE3C5C29E4,180,1,0.001069,131,69,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,0,0,0,-1,-1,-1,1,VTS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96179,2013-01-01 06:56:07,3541431521954716025,15656085761070693788,448AAEE57B1B54724E9F3221802FB0C4,650,1,0.066301,56,73,-1.0,-1.0,-1.0,3.350194,3.156692,2.863022,0,0,0,51,143,288,4,CMT
96180,2013-01-01 06:58:35,3541431521954640956,12194716708888593583,C8DD320CEEFFEFABA44915D4CDF22A3F,650,1,0.051151,75,48,-1.0,-1.0,-1.0,2.633964,2.813232,2.775777,0,0,0,71,198,223,1,CMT
96181,2013-01-01 07:02:20,3541431521954661961,15739675127705139645,F3866CC52068990BFA8AE940ED4E989D,650,1,0.000038,-66,64,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,0,0,0,-1,-1,-1,1,CMT
96182,2013-01-01 16:19:12,3541431521954676515,5126275902684945324,92E0FFA20B5A5FDBFA9C484BD1E85914,650,1,0.042897,73,39,-1.0,-1.0,-1.0,2.232464,2.654692,2.444455,0,0,0,24,85,195,1,CMT


In [15]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

In [16]:
X = examples.dataframe.drop(["target", "_key", "_key_hash", "_subsort", "_time"], axis=1)
y = examples.dataframe["target"]


columns_to_encode = ['vendor']
ct = ColumnTransformer(
    [('encoder', OneHotEncoder(sparse=False), columns_to_encode)],
    remainder='passthrough' 
)
X = ct.fit_transform(X)

Xtr, Xv, ytr, yv = train_test_split(X, y, train_size=50000, test_size=10000, random_state=42)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# Try different parameters! My favorite is random search :)
xgb_pars = {'min_child_weight': 50, 'eta': 0.3, 'colsample_bytree': 0.7, 'max_depth': 5,
            'subsample': 0.7, 'lambda': 1., 'nthread': 4, 'booster' : 'gbtree',
            'eval_metric': 'rmsle', 'objective': 'reg:linear'}

In [17]:
# You could try to train with more epoch
# 0.28976 leader, 0.37195 10%
model = xgb.train(xgb_pars, dtrain, 60, watchlist, early_stopping_rounds=50,
                  maximize=False, verbose_eval=10)

[12:27:53] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmsle:1.18370	valid-rmsle:1.17266
[10]	train-rmsle:0.56332	valid-rmsle:0.56635
[20]	train-rmsle:0.55745	valid-rmsle:0.56367
[30]	train-rmsle:0.55421	valid-rmsle:0.56307
[40]	train-rmsle:0.55128	valid-rmsle:0.56377
[50]	train-rmsle:0.54971	valid-rmsle:0.56464
[54]	train-rmsle:0.54875	valid-rmsle:0.56428
